In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import BayesianRidge
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Loading open source dataset with employees information

employees = pd.read_csv('employees.csv')
employees.head(20)

,First Name,Gender,Salary,Bonus %,Senior Management,Team
0,Douglas,Male,97308,6.945,TRUE,Marketing
1,Thomas,Male,61933,NaN,TRUE,NaN
2,Maria,Female,130590,11.858,FALSE,Finance
3,Jerry,Male,NaN,9.34,TRUE,Finance
4,Larry,Male,101004,1.389,TRUE,Client Services
5,Dennis,n.a.,115163,10.125,FALSE,Legal
6,Ruby,Female,65476,10.012,TRUE,Product
7,NaN,Female,45906,11.598,NaN,Finance
8,Angela,NaN,NaN,18.523,TRUE,Engineering
9,Frances,Female,139852,7.524,TRUE,Business Development


In [3]:
# Checking types of columns

employees.dtypes

First Name           object
Gender               object
Salary               object
Bonus %              object
Senior Management    object
Team                 object
dtype: object

In [4]:
# Basic stats

employees.describe()

,First Name,Gender,Salary,Bonus %,Senior Management,Team
count,931,852,998,997,932,957
unique,201,3,993,968,4,13
top,Marilyn,Female,91462,8.999,TRUE,Client Services
freq,11,428,2,3,467,105


In [5]:
# Checking Salary column

print('Salary')
employees['Salary'].head(10)

Salary


0     97308
1     61933
2    130590
3       NaN
4    101004
5    115163
6     65476
7     45906
8       NaN
9    139852
Name: Salary, dtype: object

In [6]:
# Checking Gender column. We can see non-standard input values

employees['Gender'].head(10)

0      Male
1      Male
2    Female
3      Male
4      Male
5      n.a.
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object

In [7]:
# Checking the number of null values 

employees.isnull().sum()

First Name            69
Gender               148
Salary                 2
Bonus %                3
Senior Management     68
Team                  43
dtype: int64

We noticed that 'n.a.' text isn't converted to NaN. We will pass these formats in the .read_csv() method for Pandas to recognize them as missing values.

In [8]:
# Creating a list of corrupt values

missing_values = ["n.a.","NA","n/a", "na", "--", "?"]
employees_with_na = pd.read_csv("employees.csv", na_values = missing_values)

# Printing gender again

print(employees_with_na['Gender'].head(10))

0      Male
1      Male
2    Female
3      Male
4      Male
5       NaN
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object


In [9]:
# Checking the number of null values again

employees_with_na.isnull().sum()

First Name            70
Gender               149
Salary                 5
Bonus %                4
Senior Management     71
Team                  48
dtype: int64

We can see that using the list of corrupt values we managed to catch data that needs attention and bring it to one standard. 

In [10]:
# Changing column names to adhere to naming conventions

employees_with_na.rename(columns={"First Name" : "first_name", "Gender" : "gender", "Salary" : "salary", "Bonus %" : "bonus", "Senior Management" : "senior_management", "Team" : "team"}, inplace=True)


### Data types correction

We can notice that columns have object type whereas Bonus % and Salary have to be numeric. 

In [11]:
# Creating a function to change data type to numeric

def make_int(i):
    try:
        return int(i)
    except:
        return np.nan

# Applying make_int function to the entire series using map

employees_with_na['salary'] = employees_with_na['salary'].map(make_int)
employees_with_na['bonus'] = employees_with_na['bonus'].map(make_int)

In [12]:
# Checking results

employees_with_na.head()

,first_name,gender,salary,bonus,senior_management,team
0,Douglas,Male,97308.0,6.0,True,Marketing
1,Thomas,Male,61933.0,NaN,True,NaN
2,Maria,Female,130590.0,11.0,False,Finance
3,Jerry,Male,NaN,9.0,True,Finance
4,Larry,Male,101004.0,1.0,True,Client Services


In [13]:
# Checking resulting data types

employees_with_na.dtypes

first_name            object
gender                object
salary               float64
bonus                float64
senior_management     object
team                  object
dtype: object

## Addressing missing values

For the purpose of further visualization we will address missing values depending on data type. 

In [14]:
# Replacing NaNs using Median/Mean of the column

employees_with_na['salary'].fillna(employees_with_na['salary'].median(), inplace=True)
employees_with_na['bonus'].fillna(employees_with_na['bonus'].median(), inplace=True)

In [15]:
employees_with_na['gender'].fillna('No Gender', inplace=True)

In [16]:
employees_with_na.head(15)

,first_name,gender,salary,bonus,senior_management,team
0,Douglas,Male,97308.0,6.0,True,Marketing
1,Thomas,Male,61933.0,9.0,True,NaN
2,Maria,Female,130590.0,11.0,False,Finance
3,Jerry,Male,90370.0,9.0,True,Finance
4,Larry,Male,101004.0,1.0,True,Client Services
5,Dennis,No Gender,115163.0,10.0,False,Legal
6,Ruby,Female,65476.0,10.0,True,Product
7,NaN,Female,45906.0,11.0,NaN,Finance
8,Angela,No Gender,90370.0,18.0,True,Engineering
9,Frances,Female,139852.0,7.0,True,Business Development


In [17]:
# Checking the number of null values again

employees_with_na.isnull().sum()

first_name           70
gender                0
salary                0
bonus                 0
senior_management    71
team                 48
dtype: int64

In [18]:
# We will drop records with missing values for the purpose of visualization

employees_without_na = employees_with_na.dropna(axis=0)

In [19]:
# Checking the number of null values in a new dataset

employees_without_na.isnull().sum()

first_name           0
gender               0
salary               0
bonus                0
senior_management    0
team                 0
dtype: int64

In [20]:
# Number of records and data types

employees_without_na.info()

<class 'pandas.core.frame.DataFrame'>
Index: 887 entries, 0 to 999
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   first_name         887 non-null    object 
 1   gender             887 non-null    object 
 2   salary             887 non-null    float64
 3   bonus              887 non-null    float64
 4   senior_management  887 non-null    object 
 5   team               887 non-null    object 
dtypes: float64(2), object(4)
memory usage: 48.5+ KB


## Saving data for future use

In [21]:
# Extracting preprocessed data to

employees_without_na.to_csv("employees_cleaned.csv", index=False)

## Dashboard using Power BI

Power BI was used to create customizable dashboard based on this data. All charts are interactive and can be filtered to display needed information.  

![alt text](employees_powerbi_dashboard.png "Power BI Dashboard overview")